In [21]:
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#################################################
transformer=transforms.Compose([
    #auf 150x150 Pixel skalieren
    transforms.Resize((28,28)),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(), # pixelrange (of each colour channel) from 0-255 to 0-; datatyp of image is now tensor,not numpy
    transforms.Normalize([0.5,0.5,0.5], #range from 0-1 to [-1,1]
                        [0.5,0.5,0.5])
]
)
##################################################################
train_path='/home/b1/CNN/DataGeneration_v1_CNN/dataset_fullBackgr/train'
test_path='/home/b1/CNN/DataGeneration_v1_CNN/dataset_fullBackgr/test'

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    # batch size should fit to the CPU-power
    batch_size=8, shuffle=True
)
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    # batch size should fit to the CPU-power
    batch_size=8, shuffle=True
)
#########################################################
class ConvNet(nn.Module):
        def __init__(self,num_classes=2):
            super(ConvNet,self).__init__()
            
            # Input shape = (256,3,150,150); batch_size,numberOfChannels(rgb),height,width
            # in_channels= rgb
           
            self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
            # Output size after conv. filter ((width-kernel_size+2*Padding)/stride) +1
            # new shape: (256,12,150,150)
            self.bn1=nn.BatchNorm2d(num_features=12)
            # new shape: (256,12,150,150)
            self.relu1=nn.ReLU()
            # new shape: (256,12,150,150)
            self.pool=nn.MaxPool2d(kernel_size=2) # reduce the image size with factor 2
            # new shape: (256,12,75,75)
            
            
            self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
            # new shape: (256,20,75,75)
            self.relu2=nn.ReLU()
            # new shape: (256,20,75,75)
            
            self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
            # new shape: (256,32,75,75)
            self.bn3=nn.BatchNorm2d(num_features=32)
            # new shape: (256,32,75,75)
            self.relu3=nn.ReLU()
            # new shape: (256,32,75,75)
            
            # last layer:
            self.fc=nn.Linear(in_features=32*14*14, out_features=num_classes)
            
            #Feed forward function
        def forward(self,input):
            output=self.conv1(input)
            output=self.bn1(output)
            output=self.relu1(output)
                
            output=self.pool(output)
                
            output=self.conv2(output)
            output=self.relu2(output)
                
            output=self.conv3(output)
            output=self.bn3(output)
            output=self.relu3(output)
                
            # output will be in matrix form, with shape (256,32,75,75)
                
            output=output.view(-1,32*14*14) #last shape
                
            output=self.fc(output)
                
            return output
###########################################################################
model=ConvNet(num_classes=2).to(device)

optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.001)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.001, weight_decay=0.1)
loss_function=nn.CrossEntropyLoss()

num_epochs=25
#size of training/testing images
train_count=len(glob.glob(train_path+'/**/*'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))
print(train_count,test_count)

6729 201


In [ ]:
best_accuracy=0.0

for epoch in range(num_epochs):
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i, (images,labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count

    # Model evaluation
    model.eval()
    
    test_accuracy=0.0
    for i, (images,labels) in enumerate(test_loader):
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
        
    test_accuracy=test_accuracy/test_count
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    #save model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy

Epoch: 0 Train Loss: tensor(0.0863) Train Accuracy: 0.9780056471986922 Test Accuracy: 0.7412935323383084
Epoch: 1 Train Loss: tensor(0.0505) Train Accuracy: 0.9860306137613315 Test Accuracy: 0.6368159203980099
Epoch: 2 Train Loss: tensor(0.0300) Train Accuracy: 0.9909347599940556 Test Accuracy: 0.6517412935323383
Epoch: 3 Train Loss: tensor(0.0240) Train Accuracy: 0.9916778124535592 Test Accuracy: 0.7313432835820896
Epoch: 4 Train Loss: tensor(0.0184) Train Accuracy: 0.993461138356368 Test Accuracy: 0.6567164179104478
Epoch: 5 Train Loss: tensor(0.0167) Train Accuracy: 0.9930153068806657 Test Accuracy: 0.8706467661691543
Epoch: 6 Train Loss: tensor(0.0078) Train Accuracy: 0.9973250111457869 Test Accuracy: 0.7412935323383084
